In [1]:
# To DO:
# Resolve column issue on index.  Is this fine?
# https://www.datahub.io/core/country-list.  Add this to sources list

# Change format on pop-up

In [109]:
import pandas as pd
import numpy as np
import os
import gmaps
import requests
import sys

from pprint import pprint

sys.path.insert(0,'../')
from config import g_key

gmaps.configure(g_key)

In [22]:
vacation_cities_df = pd.read_csv(os.path.join("..","vacation_search","WeatherPy_vacation.csv")).drop(["Unnamed: 0"], axis=1)
vacation_cities_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Plettenberg Bay,ZA,-34.0527,23.3716,65.46,88,19,5.32,few clouds,Thanda Vista B&B
1,Atuona,PF,-9.8000,-139.0333,77.95,77,11,16.02,few clouds,Banque Socredo
2,Carnarvon,AU,-24.8667,113.6333,75.27,78,0,13.80,clear sky,Hospitality Carnarvon
3,Lompoc,US,34.6391,-120.4579,61.57,73,20,12.66,few clouds,"Holiday Inn Express Lompoc, an IHG Hotel"
4,Araguaina,BR,-7.1911,-48.2072,72.57,96,100,6.20,light rain,Hotel São Vicente
...,...,...,...,...,...,...,...,...,...,...
57,Marsh Harbour,BS,26.5412,-77.0636,77.67,63,0,0.83,clear sky,Abaco Beach Resort
58,Atambua,ID,-9.1061,124.8925,73.56,89,90,1.34,overcast clouds,Pegadaian Betun
59,Vanimo,PG,-2.6741,141.3028,77.59,91,100,4.54,overcast clouds,Vahmoneh Lodge Vanimo
60,Bredasdorp,ZA,-34.5322,20.0403,63.72,63,100,11.50,overcast clouds,Bredasdorp Country Manor


In [26]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_cities_df.iterrows()]

coordinates_df = vacation_cities_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure(center=(0.0, 0.0), 
                   zoom_level=1.5,
                   layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig

Figure(layout=FigureLayout(height='500px', width='800px'))

In [45]:
country_code_map_df = pd.read_csv(os.path.join("..","weather_database","country_code_map.csv"))
country_code_map_df

,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
244,Wallis and Futuna,WF
245,Western Sahara,EH
246,Yemen,YE
247,Zambia,ZM


In [47]:
# Think I might go with Australia at least with sample data set, but let's run the numbers:
country_counts_df = pd.DataFrame(data=vacation_cities_df.value_counts("Country"))

country_counts_df.columns = ["Count"]

country_counts_df
# eligible_countries.loc[eligible_countries[0] >= 4]

,Count
Country,
AU,11
ZA,7
US,7
BR,5
ID,4
CL,3
IN,2
PF,2
NZ,2


In [53]:
eligible_countries_df = pd.merge(left=country_counts_df,
                                 right=country_code_map_df,
                                 left_on = "Country",
                                 right_on = "Code",
                                 how="inner")

eligible_countries_df = eligible_countries_df.loc[(eligible_countries_df["Count"] >= 4)]

eligible_countries_df

,Count,Name,Code
0,11,Australia,AU
1,7,South Africa,ZA
2,7,United States,US
3,5,Brazil,BR
4,4,Indonesia,ID


In [68]:
# do input later, proof of concept!
print("Please choose a country code using the above list of eligible countries: ")

chosen_country_code = 'AU'
chosen_country_name = eligible_countries_df.loc[eligible_countries_df["Code"] == chosen_country_code]["Name"].get(key=0)
chosen_country_count = eligible_countries_df.loc[eligible_countries_df["Code"] == chosen_country_code]["Count"].get(key=0)

print(f"You have chosen {chosen_country_name}, which has {chosen_country_count} eligible cities: ")

chosen_country_cities_df = vacation_cities_df.loc[vacation_cities_df["Country"] == chosen_country_code]
chosen_country_cities_df

Please choose a country code using the above list of eligible countries: 
You have chosen Australia, which has 11 eligible cities: 


,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
2,Carnarvon,AU,-24.8667,113.6333,75.27,78,0,13.80,clear sky,Hospitality Carnarvon
7,Mount Gambier,AU,-37.8333,140.7667,63.09,57,100,4.70,overcast clouds,The Commodore
9,New Norfolk,AU,-42.7826,147.0587,56.71,81,0,3.27,clear sky,The Shingles Riverside Cottages
12,Hobart,AU,-42.8794,147.3294,62.11,88,0,5.75,clear sky,Mantra on Collins Hobart
19,Busselton,AU,-33.6500,115.3333,54.09,81,26,11.77,scattered clouds,Observatory Guest House
23,Albany,AU,-35.0169,117.8917,59.43,72,90,6.91,overcast clouds,Dog Rock Motel
39,Port Lincoln,AU,-34.7333,135.8667,71.31,56,100,5.97,overcast clouds,Port Lincoln Hotel
41,Perth,AU,-31.9333,115.8333,69.67,67,75,8.05,broken clouds,Beatty Lodge
49,Launceston,AU,-41.4500,147.1667,58.64,84,6,0.00,clear sky,Hotel Grand Chancellor Launceston
50,Port Macquarie,AU,-31.4333,152.9167,65.89,83,100,3.53,overcast clouds,Rydges Port Macquarie


In [70]:
city_name_1 = "Mount Isa"
city_name_2 = "Port Macquarie"
city_name_3 = "Mount Gambier"
city_name_4 = "Port Lincoln"

# convert each chosen city to a dataframe
vacation_start = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_1)]
vacation_end = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_1)]
vacation_stop1 = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_2)]
vacation_stop2 = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_3)]
vacation_stop3 = chosen_country_cities_df.loc[(chosen_country_cities_df["City"] == city_name_4)]

In [93]:
# Get the lat-long pairs
start = vacation_start[["Lat", "Long"]].to_numpy()
end = vacation_end[["Lat", "Long"]].to_numpy()
stop1 = vacation_stop1[["Lat", "Long"]].to_numpy()
stop2 = vacation_stop2[["Lat", "Long"]].to_numpy()
stop3 = vacation_stop3[["Lat", "Long"]].to_numpy()

start[0][0]

-20.7333

In [102]:
fig = gmaps.figure()
chosen_trip = gmaps.directions_layer(
         start=(start[0][0],start[0][1]),
         end=(end[0][0],end[0][1]),
         waypoints=[(stop1[0][0],stop1[0][1]),
                    (stop2[0][0],stop2[0][1]),
                    (stop3[0][0],stop3[0][1])],
        optimize_waypoints=True
)
fig.add_layer(chosen_trip)
fig

Figure(layout=FigureLayout(height='420px'))

In [103]:
itinerary_df = pd.concat(objs=[vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],
                         ignore_index=True)
itinerary_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description,Hotel Name
0,Mount Isa,AU,-20.7333,139.5000,78.57,57,6,0.00,light rain,ibis Styles Mt Isa Verona
1,Port Macquarie,AU,-31.4333,152.9167,65.89,83,100,3.53,overcast clouds,Rydges Port Macquarie
2,Mount Gambier,AU,-37.8333,140.7667,63.09,57,100,4.70,overcast clouds,The Commodore
3,Port Lincoln,AU,-34.7333,135.8667,71.31,56,100,5.97,overcast clouds,Port Lincoln Hotel


In [107]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

coordinates_df = itinerary_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure( #center=(0.0, 0.0), 
                   # zoom_level=1.5,
                   layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig

Figure(layout=FigureLayout(height='500px', width='800px'))